In [41]:
import requests
import os
import pandas as pd
from bs4 import BeautifulSoup
from mtranslate import translate
import http.cookies
import re
import json

In [2]:
cookies= [
{
    "domain": ".taobao.com",
    "expirationDate": 1648928883.616592,
    "hostOnly": False,
    "name": "_cc_",
    "path": "/",
    "sameSite": "no_restriction",
    "secure": True,
    "session": False,
    "storeId": "0",
    "value": "V32FPkk%2Fhw%3D%3D",
    "id": 1
},
{
    "domain": ".taobao.com",
    "expirationDate": 1617983169.891207,
    "hostOnly": False,
    "httpOnly": False,
    "name": "_m_h5_tk",
    "path": "/",
    "sameSite": "no_restriction",
    "secure": True,
    "session": False,
    "storeId": "0",
    "value": "96bb7c9d067d6cf0854dda61ffd7c7db_1617388089917",
    "id": 2
},
{
    "domain": ".taobao.com",
    "expirationDate": 1617983169.891318,
    "hostOnly": False,
    "httpOnly": False,
    "name": "_m_h5_tk_enc",
    "path": "/",
    "sameSite": "no_restriction",
    "secure": True,
    "session": False,
    "storeId": "0",
    "value": "78751c2fae1aac9d6a75e3771079ee4a",
    "id": 3
},
{
    "domain": ".taobao.com",
    "hostOnly": False,
    "httpOnly": True,
    "name": "_samesite_flag_",
    "path": "/",
    "sameSite": "no_restriction",
    "secure": True,
    "session": True,
    "storeId": "0",
    "value": "True",
    "id": 4
},
{
    "domain": ".taobao.com",
    "hostOnly": False,
    "httpOnly": False,
    "name": "_tb_token_",
    "path": "/",
    "sameSite": "unspecified",
    "secure": False,
    "session": True,
    "storeId": "0",
    "value": "e7593b4e9b036",
    "id": 5
},
{
    "domain": ".taobao.com",
    "expirationDate": 2246515695,
    "hostOnly": False,
    "httpOnly": False,
    "name": "cna",
    "path": "/",
    "sameSite": "no_restriction",
    "secure": True,
    "session": False,
    "storeId": "0",
    "value": "G7uZGLjxwEUCAWcHT1zZjb8h",
    "id": 6
},
{
    "domain": ".taobao.com",
    "hostOnly": False,
    "httpOnly": True,
    "name": "cookie2",
    "path": "/",
    "sameSite": "no_restriction",
    "secure": True,
    "session": True,
    "storeId": "0",
    "value": "13f346f295220f1a83ceb0d89d9f0da7",
    "id": 7
},
{
    "domain": ".taobao.com",
    "hostOnly": False,
    "httpOnly": False,
    "name": "csg",
    "path": "/",
    "sameSite": "no_restriction",
    "secure": True,
    "session": True,
    "storeId": "0",
    "value": "e89aa674",
    "id": 8
},
{
    "domain": ".taobao.com",
    "hostOnly": False,
    "httpOnly": False,
    "name": "dnk",
    "path": "/",
    "sameSite": "no_restriction",
    "secure": True,
    "session": True,
    "storeId": "0",
    "value": "bsonmez",
    "id": 9
},
{
    "domain": ".taobao.com",
    "expirationDate": 1932724124.732321,
    "hostOnly": False,
    "httpOnly": True,
    "name": "enc",
    "path": "/",
    "sameSite": "no_restriction",
    "secure": True,
    "session": False,
    "storeId": "0",
    "value": "8jttWMyc8m2LA4SNj24Q%2FiMMHUE1WEE4IYG%2BICg82ZTxcuRkw8xrFgc%2BFxJE60%2BVJUQWzehAp6HwhBMS00%2Fp3Q%3D%3D",
    "id": 10
},
{
    "domain": ".taobao.com",
    "hostOnly": False,
    "httpOnly": False,
    "name": "existShop",
    "path": "/",
    "sameSite": "no_restriction",
    "secure": True,
    "session": True,
    "storeId": "0",
    "value": "MTYxNzM2NDA4Mw%3D%3D",
    "id": 11
},
{
    "domain": ".taobao.com",
    "expirationDate": 1647360502.425483,
    "hostOnly": False,
    "httpOnly": False,
    "name": "hng",
    "path": "/",
    "sameSite": "no_restriction",
    "secure": True,
    "session": False,
    "storeId": "0",
    "value": "GLOBAL%7Czh-CN%7CUSD%7C999",
    "id": 12
},
{
    "domain": ".taobao.com",
    "expirationDate": 1632933294,
    "hostOnly": False,
    "httpOnly": False,
    "name": "isg",
    "path": "/",
    "sameSite": "no_restriction",
    "secure": True,
    "session": False,
    "storeId": "0",
    "value": "BGhoxPxnnGwZV7Do1uKq_01LOVZ6kcybGXD-iyKZtOPWfQjnyqGcK_63cQ2NzYRz",
    "id": 13
},
{
    "domain": ".taobao.com",
    "expirationDate": 1632933294,
    "hostOnly": False,
    "httpOnly": False,
    "name": "l",
    "path": "/",
    "sameSite": "unspecified",
    "secure": False,
    "session": False,
    "storeId": "0",
    "value": "eBata1gljF97hJtBBOfwourza77OSIRAguPzaNbMiOCP9a5e528FW6ZoD2YwC3MNh6m6R3RatrsuBeYBqCbk3CPMrNWQtWDmn",
    "id": 14
},
{
    "domain": ".taobao.com",
    "expirationDate": 1619984883.616251,
    "hostOnly": False,
    "httpOnly": False,
    "name": "lgc",
    "path": "/",
    "sameSite": "no_restriction",
    "secure": True,
    "session": False,
    "storeId": "0",
    "value": "bsonmez",
    "id": 15
},
{
    "domain": ".taobao.com",
    "expirationDate": 1617452091.045847,
    "hostOnly": False,
    "httpOnly": False,
    "name": "lLtC1_",
    "path": "/",
    "sameSite": "no_restriction",
    "secure": True,
    "session": False,
    "storeId": "0",
    "value": "1",
    "id": 16
},
{
    "domain": ".taobao.com",
    "expirationDate": 1703408132.303427,
    "hostOnly": False,
    "httpOnly": False,
    "name": "miid",
    "path": "/",
    "sameSite": "no_restriction",
    "secure": True,
    "session": False,
    "storeId": "0",
    "value": "1023263651835633479",
    "id": 17
},
{
    "domain": ".taobao.com",
    "expirationDate": 1618011969.629743,
    "hostOnly": False,
    "httpOnly": False,
    "name": "mt",
    "path": "/",
    "sameSite": "no_restriction",
    "secure": True,
    "session": False,
    "storeId": "0",
    "value": "ci=-1_0",
    "id": 18
},
{
    "domain": ".taobao.com",
    "expirationDate": 1648928883.616371,
    "hostOnly": False,
    "httpOnly": False,
    "name": "sgcookie",
    "path": "/",
    "sameSite": "no_restriction",
    "secure": True,
    "session": False,
    "storeId": "0",
    "value": "E100FrG8orBYLQc4DlNISjY%2BUnBR44whONftQ%2FGkqgZHVdeiO7gTTo5UJFQmFrsTHNNEhZt3ibEaoW%2Bb%2FRCv8ZNg9A%3D%3D",
    "id": 19
},
{
    "domain": ".taobao.com",
    "hostOnly": False,
    "httpOnly": True,
    "name": "skt",
    "path": "/",
    "sameSite": "no_restriction",
    "secure": True,
    "session": True,
    "storeId": "0",
    "value": "4171cecb81173d14",
    "id": 20
},
{
    "domain": ".taobao.com",
    "expirationDate": 1625168883.616306,
    "hostOnly": False,
    "httpOnly": False,
    "name": "t",
    "path": "/",
    "sameSite": "no_restriction",
    "secure": True,
    "session": False,
    "storeId": "0",
    "value": "7f7b085e7ba1473ab752117398ab827c",
    "id": 21
},
{
    "domain": ".taobao.com",
    "expirationDate": 1632933295,
    "hostOnly": False,
    "httpOnly": False,
    "name": "tfstk",
    "path": "/",
    "sameSite": "unspecified",
    "secure": False,
    "session": False,
    "storeId": "0",
    "value": "cnCPBjMTz7FrfrdQBQOEAjRQc9KRZk5h18LpEtmR3XCqMh9lidipohC52MoADLf..",
    "id": 22
},
{
    "domain": ".taobao.com",
    "expirationDate": 1647331692.619293,
    "hostOnly": False,
    "httpOnly": False,
    "name": "thw",
    "path": "/",
    "sameSite": "unspecified",
    "secure": False,
    "session": False,
    "storeId": "0",
    "value": "xx",
    "id": 23
},
{
    "domain": ".taobao.com",
    "hostOnly": False,
    "httpOnly": False,
    "name": "tk_trace",
    "path": "/",
    "sameSite": "no_restriction",
    "secure": True,
    "session": True,
    "storeId": "0",
    "value": "oTRxOWSBNwn9dPyorMJE%2FoPdY8zMG1aAN%2F0TkjYGZjkj6rrK3kv4LgxGhtlxvv2n251iRZBLi5%2BxLVnyk1ChKBNHHMloZjvW0QOFCwsMXbRJC3R4c7Eo8ay6GQ28KjbKOJA0xJL0vqpkzM%2FJ49QqUWjYc8LQsdUwhf1TkCJ5%2FJBcL6wxEdAKBAz6eJ5ZUrX7H34gi4xLSiSiBwUvfNGHprgfqQ7llZznY4xy68C6rDBNHNSqwVVzR4zN79tg5%2FKqcyo3rOjc%2F4sFFD0UjHh3MWwaULb%2FSyCz1IX8HChenhIZAvFwn9esrTcoHTCbPh4c%2Fev8bA%2Be442ZTIcngoBs5dhgZUINJvp94nxUdR1PzyEt%2Bq4ZEGVxu50%3D",
    "id": 24
},
{
    "domain": ".taobao.com",
    "expirationDate": 1648928883.616543,
    "hostOnly": False,
    "httpOnly": False,
    "name": "tracknick",
    "path": "/",
    "sameSite": "no_restriction",
    "secure": True,
    "session": False,
    "storeId": "0",
    "value": "bsonmez",
    "id": 25
},
{
    "domain": ".taobao.com",
    "hostOnly": False,
    "httpOnly": False,
    "name": "uc1",
    "path": "/",
    "sameSite": "no_restriction",
    "secure": True,
    "session": True,
    "storeId": "0",
    "value": "cookie14=Uoe1hdI30%2BOTmA%3D%3D&pas=0&cookie16=VT5L2FSpNgq6fDudInPRgavC%2BQ%3D%3D&cookie21=VT5L2FSpdiBh&existShop=False",
    "id": 26
},
{
    "domain": ".taobao.com",
    "expirationDate": 1619984883.614104,
    "hostOnly": False,
    "httpOnly": True,
    "name": "uc3",
    "path": "/",
    "sameSite": "no_restriction",
    "secure": True,
    "session": False,
    "storeId": "0",
    "value": "lg2=Vq8l%2BKCLz3%2F65A%3D%3D&vt3=F8dCuAtYSOHRFoHxQ6c%3D&nk2=AResYV1KHQ%3D%3D&id2=UUpmnchHEx92pg%3D%3D",
    "id": 27
},
{
    "domain": ".taobao.com",
    "expirationDate": 1619984883.616491,
    "hostOnly": False,
    "httpOnly": True,
    "name": "uc4",
    "path": "/",
    "sameSite": "no_restriction",
    "secure": True,
    "session": False,
    "storeId": "0",
    "value": "id4=0%40U2gsEZG7xJIkbBNmqi%2B8VAr%2F%2BNrF&nk4=0%40A7qxXvSLjvhafI0cBGt15YH1",
    "id": 28
},
{
    "domain": ".taobao.com",
    "hostOnly": False,
    "httpOnly": False,
    "name": "v",
    "path": "/",
    "sameSite": "unspecified",
    "secure": False,
    "session": True,
    "storeId": "0",
    "value": "0",
    "id": 29
},
{
    "domain": ".taobao.com",
    "expirationDate": 1617448232,
    "hostOnly": False,
    "httpOnly": False,
    "name": "xlly_s",
    "path": "/",
    "sameSite": "no_restriction",
    "secure": True,
    "session": False,
    "storeId": "0",
    "value": "1",
    "id": 30
}
]

In [3]:
headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/89.0.4389.90 Safari/537.36 Edg/89.0.774.63',
#           'cna': "G7uZGLjxwEUCAWcHT1zZjb8h",
# 'gmkey': "EXP",
# 'gokey': "msg%3Dmodule.init%257Cbat%257Cmui%252Fzebra-oversea-seo-category%257C4.0.0%257C%257C690%257C0%257C1%253Bbat%257C%257C%257C%257C%257C0%257C0%257C1%253Bdynamic.init%257CzebraDynamic%257Cmui%252Fzebra-dynamic%257C4.2.12%257C%257C647%257C0%257C6%253Bdynamic.parse%257CzebraDynamic%257Cmui%252Fzebra-dynamic%257C4.2.12%257C%257C640%257C0%257C6%253Brequest.init%257Cdynamic.parse%257Cmui%252Fzebra-dynamic%257C4.2.12%257C%257C9%257C0%257C6%253Btmall-dynamic.init%257CtmallDynamic%257Cmui%252Ftmall-dynamic%257C4.2.1%257C%257C632%257C0%257C3%26hash%3D1512786519%26spm%3Da21wu.241046%26client%3Dpc%26token%3DlyUgVnwP5sC63GI2SVfe%26proxy%3D%26_hng%3DGLOBAL%25257Czh-CN%25257CUSD%25257C999%26aws%3D1%26jsver%3Daplus_std%26lver%3D8.15.4%26pver%3D0.7.11%26cache%3Dd28039b%26page_cna%3DG7uZGLjxwEUCAWcHT1zZjb8h%26_slog%3D0",
# 'logtype': "2",
# 'spm-cnt': "a21wu.241046-global.0.0.41cab6cbhAOsC9",
# '_p_url': "https%3A%2F%2Fworld.taobao.com%2F",
          }

In [7]:
sess = requests.Session()

In [8]:
for cookie in cookies:
    sess.cookies.set(cookie['name'], cookie['value'])

In [55]:
site = sess.get('https://s.taobao.com/search?spm=a21wu.241046-global.6977698868.164.4103b6cbbgbVCY&q=%E9%A5%B0%E5%93%81DIY&acm=lb-zebra-241046-2075394.1003.4.1812236&scm=1003.4.lb-zebra-241046-2075394.OTHER_15206383875223_1812236',
                   headers=headers,)

In [56]:
soup = BeautifulSoup(site.content)

In [57]:
def get_items(res):
    g_page_config = re.search(r"g_page_config = (.*?);\n", res.text)
    page_config_json = json.loads(g_page_config.group(1))
    page_items = page_config_json['mods']['itemlist']['data']['auctions']
    result = []
    for each_item in page_items:
        dict1 = dict.fromkeys(('nid','title','detail_url','view_price','view_sales','nick'))
        dict1['nid'] = each_item['nid']
        dict1['title'] = each_item['title']
        dict1['detail_url'] = "https:"+each_item['detail_url']
        dict1['view_price'] = each_item['view_price']
        dict1['view_sales'] = each_item['view_sales']
        dict1['nick'] = each_item['nick']
        result.append(dict1)
    return result

In [58]:
get_items(site)

[{'nid': '624327944034',
  'title': 'ZENGLIU设计师拼件925纯银字母吊坠手链<span class=H>DIY</span>串珠时尚情侣配件<span class=H>饰品</span>',
  'detail_url': 'https://detail.tmall.com/item.htm?id=624327944034&ns=1&abbucket=5',
  'view_price': '69.00',
  'view_sales': '33人付款',
  'nick': 'zengliu旗舰店'},
 {'nid': '622104997017',
  'title': '卡蒂罗&故宫上新 御猫项链女纯银锁骨链轻奢首<span class=H>饰品</span>新款生日礼物',
  'detail_url': 'https://detail.tmall.com/item.htm?id=622104997017&ad_id=&am_id=&cm_id=140105335569ed55e27b&pm_id=&abbucket=5#detail',
  'view_price': '689.00',
  'view_sales': '1人付款',
  'nick': '卡蒂罗旗舰店'},
 {'nid': '632069453704',
  'title': '水晶碧玺紫水晶发晶吊坠项链手串手链<span class=H>DIY</span><span class=H>饰品</span>',
  'detail_url': 'https://detail.tmall.com/item.htm?id=632069453704&ns=1&abbucket=5',
  'view_price': '100.00',
  'view_sales': '4195人付款',
  'nick': '同圣珠宝旗舰店'},
 {'nid': '637986098692',
  'title': '雨嘉珠宝琥珀蜜蜡回流古董老蜜蜡扣子项链毛衣链<span class=H>diy</span>项链颈<span class=H>饰品</span>女',
  'detail_url': 'https://item.taobao.com/item.ht

In [59]:
site1 = sess.get('https://detail.tmall.com/item.htm?id=631135729332&ad_id=&am_id=&cm_id=140105335569ed55e27b&pm_id=&abbucket=5#detail')

In [60]:
soup = BeautifulSoup(site1.content)

In [61]:
soup

<!DOCTYPE html>
<html>
<head>
<meta charset="utf-8"/>
<meta content="webkit" name="renderer"/>
<meta content="IE=Edge" http-equiv="X-UA-Compatible"/>
<script>
    (function(w,d){
    try{
        var l,url='//mdskip.taobao.com/core/initItemDetail.htm?isUseInventoryCenter=false&cartEnable=true&service3C=false&isApparel=false&isSecKill=false&tmallBuySupport=true&isAreaSell=false&tryBeforeBuy=false&offlineShop=false&itemId=631135729332&showShopProm=false&isPurchaseMallPage=false&itemGmtModified=1617400089000&isRegionLevel=false&household=false&sellerPreview=false&queryMemberRight=true&addressLevel=2&isForbidBuyItem=false',isg=document.cookie.match('(^|;) ?l=([^;]*)(;|$)'),isg2 =document.cookie.match('(^|;) ?isg=([^;]*)(;|$)');
        if(!url){return;}
        var arr=["callback=setMdskip","timestamp="+(+new Date()),"isg="+(isg&&isg[2]),"isg2="+(isg2&&isg2[2])],reg=/[?&^](ip|campaignId|key|abt|cat_id|q|u_channel|areaId|sdShopId)=([^&]+)/g,params=w.location.search;
        while(r=reg.exec

In [62]:
soup.find('textarea', {'class':'ks-datalazyload'})

<textarea class="ks-datalazyload">
                <div class="bd">
            <div class="shop-rate" data-spm="d4918101">
                                    <h4>店铺动态评分<!--盟主aaa-->
                        <span class="compare">与同行业相比</span>
                    </h4>
                    <ul>
                        <li>
                            描述相符：<a href="//rate.taobao.com/user-rate-UvCIYMGHLOm8uvNTT.htm" target="_blank">
                                                            <em class="count" title="4.8分">4.8</em>
                                                    <span class="rateinfo" title="计算规则:(同行业平均分-店铺得分)/(同行业平均分-同行业店铺最低得分)">
                                                                                                                    <b class="lower"></b><em class="lower">0.59%</em>
                                                                                                            </span>
                                                    </a>
      